# Debug notebook
This notebook is only used to troubleshoot the Landsat ARD workflow processor.

Level-2 Landsat products can be ordered through [USGS ESPA](https://espa.cr.usgs.gov/ordering/new/). Orders can be automated by means of the search & discover notebooks in this same repo.

## Landsat ARD workflow

In [ ]:
import json
from utils.prepLS import prepareLS

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    logger.info("%s: %s" % (x, loaded_json[x]))
    prepareLS(**loaded_json)

## Landsat ARD job queue processor

In [ ]:
import os
import logging
import rediswq

level = os.getenv("LOGLEVEL", "INFO").upper()
logging.basicConfig(format="%(asctime)s %(levelname)-8s %(name)s %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=level)

host = os.getenv("REDIS_SERVICE_HOST", "redis-master")
q = rediswq.RedisWQ(name="jobLS", host=host)

logger = logging.getLogger("worker")
logger.info(f"Worker with sessionID: {q.sessionID()}")
logger.info(f"Initial queue state: empty={q.empty()}")

In [ ]:
import datetime

while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        logger.info(f"Working on {itemstr}")

        start = datetime.datetime.now().replace(microsecond=0)
        process_scene(itemstr)
        q.complete(item)
        end = datetime.datetime.now().replace(microsecond=0)
        logger.info(f"Total processing time {end - start}")
    else:
        logger.info("Waiting for work")

In [ ]:
logger.info("Queue empty!")